**классификатор текстов LSTM + W2V**

Евгений Борисов <esborisov@sevsu.ru>

## Данные

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()        
pd.options.display.max_colwidth = 200 

In [2]:
ff = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount','unk']

data = pd.concat([
    pd.read_csv('data/positive.csv.gz',sep=';',header=None),
    pd.read_csv('data/negative.csv.gz',sep=';',header=None),
])

data.columns = ff

data = data[['id','ttext', 'ttype']]

print( 'negative:', len(data.query('ttype==-1')), '\npositive:',len(data.query('ttype==1')) )

data.sample(10)

negative: 111923 
positive: 114911


,id,ttext,ttype
1250,409190447953608704,"@kitsune__2 да :""C за мгимо вообще обидно(",-1
107037,424218057662160896,Продолжаем серию обзоров мобильных приложений @habrahabr_ru. Сегодня анализируем приложение #Android “ЖД билеты” (,-1
47835,414972176337747968,Доброе утро всем!\nа у меня война миров в домашн.х условиях(,-1
47766,410010558314729473,@_jumbie да ради Бога )) двери моего дома всегда открыты ))) заодно Эппл повидаете )),1
1173,408919513472958464,Блиин ну я не могу)))вы бы видели как он пропархал)))шёл и так миленько ручками размахивал:D мне это реально даже не описать факчертвозьми,1
12897,409318461626933249,Слепили с бабушкой пельмени)) http://t.co/Bvsm6CgwVN,1
70536,417744070283640832,Отпусти и забудь что прошло уже не вернуть((((,-1
36919,413698466481967104,"А я всегда считала его собакой( ""@afishavozduh: парад лучших драконов в мировом кинематографе http://t.co/cjg8aqP2Gh http://t.co/GlH2Ny9NYN""",-1
74146,418624699149852672,"@liyaholmatovaaa @kamila_maf @Karinabobr в 13:30,но мне в 2 надо дома быыыть((",-1
78987,410726736406396929,"@MashaCet22 напиши моё имя с большой буквы, аахах)))))",1


In [3]:
import re

In [4]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def string_normalizer(s):
    pat = [
      #[r'ё','е'] # замена ё для унификации
      #,[r'</?[a-z]+>',' '] # удаляем xml
      [r'[:;]-*[)D]',' радость ']
      ,[r'\)\)\)*',' радость ']
      ,[r'[:;]\*',' поцелуй ']
      ,[r':\(',' печаль ']
      ,[r'\(\(\(*',' печаль ']
    ]
    return replace_patterns(s,pat).strip()

In [5]:
data['ttext_'] = data['ttext'].progress_apply(string_normalizer)

  0%|          | 0/226834 [00:00<?, ?it/s]

In [6]:
# data

In [7]:
from nltk.tokenize import word_tokenize as nltk_tokenize_word

In [8]:
def tokenize(line): # разбиваем предложения на слова
    return    [ 
        t.lower() 
        for t in nltk_tokenize_word(line) 
        if re.match(r'[а-я -]+',t.lower()) and len(t)>1 
        #      if not( (t.lower() in stopwords) or (len(t)<3) ) 
    ] 

In [9]:
data['ttext_'] = data['ttext_'].progress_apply(tokenize)

  0%|          | 0/226834 [00:00<?, ?it/s]

In [10]:
data.sample(10)

,id,ttext,ttype,ttext_
93364,422351126676398080,НЕ.МОГУ.ПОВЕРИТЬ.ЧТО.ЗАВТРА.В.ШКОЛУ!!!:(((((((((,-1,"[не.могу.поверить.что.завтра.в.школу, печаль, печаль]"
90952,410849647352373249,@Alena40477060 Всё ещё должно быть в снежинках- красиво переливаются)))))Аленка- слаааадко спать))))),1,"[всё, ещё, должно, быть, снежинках-, красиво, переливаются, радость, аленка-, слаааадко, спать, радость]"
112838,411211923578187776,"@din_thomas_ Не знаю, я всегда ленту читаю. Просто в откликах странно сидеть.\nНу и понятно почему я сразу во все переписки влезаю)",1,"[не, знаю, всегда, ленту, читаю, просто, откликах, странно, сидеть, ну, понятно, почему, сразу, во, все, переписки, влезаю]"
60001,416436849142272001,"@craazyyymofo сочувствую((\nя на них тоже не ходила, из-за музыкалки...\nи всем, по-моему, тоже было пофиг",-1,"[сочувствую, печаль, на, них, тоже, не, ходила, из-за, музыкалки, всем, по-моему, тоже, было, пофиг]"
70817,410471617253605376,"Моя знакомая бывшая трахалась в 13 лет с мужиком, которому 24:-) у него семья, дети, она еще его засадила, обвинила его в изнасиловании",1,"[моя, знакомая, бывшая, трахалась, лет, мужиком, которому, радость, него, семья, дети, она, еще, его, засадила, обвинила, его, изнасиловании]"
26298,409574678269685760,Готовила вечером мороженое:) http://t.co/JpzgVaZKhH,1,"[готовила, вечером, мороженое, радость]"
39395,413834919400706048,"Одна пара и то общество, о госпади зачем мы едем :(",-1,"[одна, пара, то, общество, госпади, зачем, мы, едем, печаль]"
83545,419907153055719424,RT @naran_official: Современные отношения((( http://t.co/NNWR0Kk8vX,-1,"[современные, отношения, печаль]"
96500,422743469673566209,@ponyashka_ Мне мама тоже самое вчера сказала :(,-1,"[мне, мама, тоже, самое, вчера, сказала, печаль]"
26595,412397817786007553,"Сижу в аэроэкспрессе на белорусской. Отправление через пять минут. Кажется, я опаздываю на самолёт. :(",-1,"[сижу, аэроэкспрессе, на, белорусской, отправление, через, пять, минут, кажется, опаздываю, на, самолёт, печаль]"


In [11]:
# data[ data['ttext_'].str.len()<1 ]
max_len = data['ttext_'].str.len().max()
max_len

35

In [12]:
data[ data['ttext_'].str.len()<1 ]

,id,ttext,ttype,ttext_
66829,410387338087632896,RT @olesyaglee: @ktyekmrf30 http://t.co/fBbOR9RPLn х),1,[]
70030,410465659223810048,RT @MilanaRepina: @LenocPlotnikova ❤спасиииибо) http://t.co/aO0vwsolYd,1,[]
77139,410705991714738176,RT @Mariya_Mila: С @ErmachonokAnton ) http://t.co/LmCxpsSoQG,1,[]
101718,411095547144306688,RT @Dasha_Jenner: @poolyasha П О Л Я \nТ Ы \nХ О Р О Ш А Я : ),1,[]
114606,411364795800354816,RT @sergey0495: http://t.co/dsvJIQP5Tgпробки 10баллов),1,[]
9374,410306860089364480,RT @_Batonchik_: @cekc_tyt @aabdullaeva1 @chemicalechelon @frank_james45 @herlocked @misty_marcie @mywhisper @to_over_kill @whoresdiefirst …,-1,[]
12183,410753661065625600,@Katyaaa_fly @Vituska1998 @daria_hey и я(,-1,[]
21396,411751245129744385,RT @SolarEclipse57: @girl_turner666 @SCOOB_JOE @Geronimo_woohoo @drinkthecyani @SolarEclipse57 @greennwood @BrianMolko_off @bananacookiee @…,-1,[]
28025,412591386681409536,@hasio_original и @whitebro_ http://t.co/dupuQSnm6D,-1,[]
29153,412648375071158272,@VasylevaMasha и я( http://t.co/GilYbA6TzZ,-1,[]


In [13]:
print( 'negative:', len(data.query('ttype==-1')), '\npositive:',len(data.query('ttype==1')) )

negative: 111923 
positive: 114911


In [14]:
data = data[ data['ttext_'].str.len()>0 ].reset_index(drop=True)
print( 'negative:', len(data.query('ttype==-1')), '\npositive:',len(data.query('ttype==1')) )

negative: 111909 
positive: 114906


https://ruscorpora.ru/new/

https://nlpub.ru/Russian_Distributional_Thesaurus    

In [15]:
import numpy as np
from gensim.models.word2vec import KeyedVectors

w2v = KeyedVectors.load_word2vec_format('w2v/all.norm-sz100-w10-cb0-it1-min100.w2v',binary=True,limit=100000)

In [16]:
# w2v.get_vector('радость').shape

In [17]:
# w2v.get_vector('школота') # Error!

In [18]:
# 'школота' in w2v
# 'радость' in w2v

In [19]:
X = [ 
 [ w2v.get_vector(w) for w in l if w in w2v ]
 for l in tqdm( data['ttext_'] ) 
]

target = data['ttype']

  0%|          | 0/226815 [00:00<?, ?it/s]

In [20]:
w2v_len = len(X[0][0])
w2v_len

100

In [21]:
z = np.zeros(w2v_len)
target = np.array([ target[i] for i,xi in enumerate(X) if len(xi)>0 ])
X = np.array([ [z]*(max_len-len(xi)) + xi for xi in X if len(xi)>0  ])
X.shape, target.shape

((226122, 35, 100), (226122,))

In [24]:
target = (target+1)//2

In [ ]:
del w2v

In [25]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( X, target, test_size=.4 )
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((135673, 35, 100), (135673,), (90449, 35, 100), (90449,))

---

---

## Библиотеки

In [ ]:
# import numpy as np
# import pandas as pd
# pd.options.display.max_colwidth = 200  
# import re
# import gc
# # import gzip
# from tqdm import tqdm
# tqdm.pandas()

## очистка данных

In [ ]:
data['ttext_clean'] = data['ttext']\
    .progress_apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )\
    .progress_apply(lambda t:[ re.sub(r'^http.*',' url ', w.strip() ) for w in t ] )\
    .progress_apply(lambda t:[ re.sub(r'^@.*',' twit ', w.strip() ) for w in t ] )\
    .progress_apply(lambda t:[ re.sub(r'[:;]-*[)D]',' happysmile ', w.strip() )for w in t ])\
    .progress_apply(lambda t:[ re.sub(r'\)\)\)*',' happysmile ', w.strip() ) for w in t ])\
    .progress_apply(lambda t:[ re.sub(r'[:;]\*',' kisssmile ', w.strip() ) for w in t ])\
    .progress_apply(lambda t:[ re.sub(r':\(',' sadsmile ', w.strip() ) for w in t ])\
    .progress_apply(lambda t:[ re.sub(r'\(\(\(*',' sadsmile ', w.strip() ) for w in t ])

In [ ]:
data['ttext_clean'] = [ ' '.join(s) for s in data['ttext_clean'] ]

In [ ]:
data['ttext_clean'] = data['ttext_clean'].str.lower()\
    .progress_apply(lambda s: re.sub( r'\W', ' ', s))\
    .progress_apply(lambda s: re.sub( r'_', ' ', s))\
    .progress_apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s))\
    .progress_apply(lambda t:[ w.strip() for w in t.split() if w.strip() ] )\
    .progress_apply(lambda t: [w for w in t if not re.match( r'\b.*\d+.*\b', w) ])

In [ ]:
data.sample(3)

In [ ]:
# удаление коротких слов
data['ttext_clean'] = data['ttext_clean'].progress_apply(lambda t:[w for w in t if len(w)>2])

In [ ]:
ppr(data)
data = data[ data['ttext_clean'].str.len()>0 ].reset_index(drop=True) 
ppr(data)

In [ ]:
data.sample(3)

In [ ]:
# voc = sorted(set.union(*[ set(s) for s in data['ttext_clean'].values.tolist() ]))
# voc = { w:i+1 for i,w in enumerate(voc) }
# voc['<pad>']= 0
# ppr(voc)

## строим датасет

### кодируем word2vec

In [ ]:
# %%time


# https://nlpub.ru/Russian_Distributional_Thesaurus
    
# from gensim.models import KeyedVectors 
# w2v_file = 'tenth.norm-sz500-w7-cb0-it5-min5.w2v'
# w2v = KeyedVectors.load_word2vec_format(w2v_file, binary=True, unicode_errors='ignore')
# w2v.init_sims(replace=True)

In [ ]:
%%time

from gensim.models.word2vec import Word2Vec

w2v_size = 128

w2v = Word2Vec( data['ttext_clean'].values, min_count=1, size=w2v_size, window=4, workers=4)

# with open('result/Word2Vec.pkl', 'wb') as f: pickle.dump(w2v, f)

In [ ]:
w2v_vocab = sorted([w for w in w2v.wv.vocab])
ppr(w2v_vocab)

In [ ]:
ii = np.random.permutation(len(w2v_vocab))[:30]
for i in ii:
    w = w2v_vocab[i]
    ww = [ v[0] for v in w2v.wv.most_similar(w, topn=5) ]
    print( w,':',ww )

In [ ]:
data['code'] = data['ttext_clean'].progress_apply(lambda t: [ w2v.wv.get_vector(w) for w in t ] )

In [ ]:
%xdel w2v
%xdel w2v_vocab

In [ ]:
code_max_len = data['code'].str.len().max()
code_max_len

In [ ]:
data['code'].str.len().describe().astype(int)

In [ ]:
z = [[0.]*w2v_size]*code_max_len
data['code'] = data['code'].progress_apply(lambda c: c+z[:(code_max_len-len(c))]  )

In [ ]:
data['code'].str.len().describe().astype(int)

In [ ]:
data['code'] = data['code'].progress_apply(np.array)

In [ ]:
x = np.stack( data['code'].values )
x.shape

In [ ]:
x = np.flip(x,axis=1)

In [ ]:
n_classes=2
target = data['ttype'].values
target = (target+1)//2
y = np.eye(n_classes)[target]

y.shape

In [ ]:
%xdel data

In [ ]:
gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=.4 )
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
y_train.sum(axis=0),y_test.sum(axis=0),

In [ ]:
%xdel x
%xdel y

In [ ]:
gc.collect()

## строим нейросеть 

In [ ]:
seq_len = x_train.shape[1]
x_train.shape , y_train.shape, seq_len, w2v_size

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(seq_len, w2v_size)))  
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_layer_names=True, show_shapes=True )

In [ ]:
%%time

hist = model.fit(x_train,y_train, batch_size=1024, epochs=15, validation_split=.3)

In [ ]:
model.evaluate(x_test ,y_test )

In [ ]:
from matplotlib import pyplot as plt

k = hist.history.keys()

w,h = 6,4

fig,ax = plt.subplots(1,len(k),figsize=(w*len(k),h))
for i,n in enumerate(k):
    ax[i].plot(hist.history[n],label=n)
    ax[i].grid(True)
    ax[i].legend()
    ax[i].set_ylim([-.1,1.1])